```python
import torch

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(3, 3, 3)
        self.conv2 = torch.nn.Conv2d(3, 3, 3)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        return x1 + x2


model = Model()
input = torch.randn(1, 3, 20, 20)
torch.onnx.export(model, input, 'temp.onnx')
```

In [28]:
import onnx
from onnx.shape_inference import infer_shapes
from onnx2torch.onnx_graph import OnnxGraph
from onnx2torch.node_converters import get_converter

onnx_model = onnx.load('../data/conv.onnx')

opset_import = {
    opsetid_proto.domain: opsetid_proto.version
    for opsetid_proto in onnx_model.opset_import
}

onnx_model = infer_shapes(onnx_model)
onnx_graph = OnnxGraph(onnx_model.graph)  # pylint: disable=no-member

for name, onnx_node in onnx_graph.nodes.items():
    version = opset_import[onnx_node.domain]
    converter = get_converter(
        domain=onnx_node.domain,
        operation_type=onnx_node.operation_type,
        version=version,
    )
    torch_module, onnx_mapping = converter(onnx_node, onnx_graph)
    print(torch_module, '\n  ', onnx_mapping)

Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1)) 
   OnnxMapping(inputs=('input',), outputs=('5',))
Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1)) 
   OnnxMapping(inputs=('input',), outputs=('6',))
OnnxBinaryMathOperation() 
   OnnxMapping(inputs=('5', '6'), outputs=('7',))
